In [2]:
! pip install pymc3

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.6/872.6 kB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 43.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pymc3 to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.2/872.2 kB 13.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 12.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 1.7 MB/s eta 0:00:00
  Crea

In [8]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt

# 假设我们有一些历史数据
history_time = np.arange(0, 10)  # 历史时间点
history_cost = np.array([80, 75, 72, 69, 68, 66, 67, 70, 74, 78])  # 历史成本
history_price = np.array([120, 118, 115, 110, 105, 100, 95, 90, 88, 85])  # 历史价格

# 定义未来时间
future_time = np.arange(10, 20)

# PyMC v4 模型
with pm.Model() as model:
    # 成本的先验：二次曲线（先降后升）
    alpha_cost = pm.Normal('alpha_cost', mu=0, sigma=10)  # 二次项
    beta_cost = pm.Normal('beta_cost', mu=0, sigma=10)  # 线性项
    gamma_cost = pm.Normal('gamma_cost', mu=70, sigma=10)  # 截距项，表示最低点大约在 70

    # 价格的先验：线性递减
    alpha_price = pm.Normal('alpha_price', mu=-5, sigma=2)  # 线性斜率
    beta_price = pm.Normal('beta_price', mu=120, sigma=10)  # 截距，初始价格

    # 成本模型
    cost_pred = alpha_cost * history_time**2 + beta_cost * history_time + gamma_cost
    # 价格模型
    price_pred = alpha_price * history_time + beta_price

    # 观察数据（历史数据作为观测值）
    cost_obs = pm.Normal('cost_obs', mu=cost_pred, sigma=5, observed=history_cost)
    price_obs = pm.Normal('price_obs', mu=price_pred, sigma=5, observed=history_price)

    # 定义未来的成本和价格预测为 Deterministic 变量
    future_cost_pred = pm.Deterministic('future_cost_pred', alpha_cost * future_time**2 + beta_cost * future_time + gamma_cost)
    future_price_pred = pm.Deterministic('future_price_pred', alpha_price * future_time + beta_price)

    # 后验采样
    trace = pm.sample(2000)

# 使用采样结果来预测未来
with model:
    # 获取未来的预测分布
    posterior_predictive = pm.sample_posterior_predictive(trace)

    # 提取未来的预测数据
    future_cost_samples = posterior_predictive['future_cost_pred']
    future_price_samples = posterior_predictive['future_price_pred']

# 预测结果可视化
plt.figure(figsize=(10, 5))

# 画出成本的预测
plt.subplot(1, 2, 1)
plt.plot(history_time, history_cost, 'b', label='历史成本')
plt.plot(future_time, future_cost_samples.mean(axis=0), 'r--', label='未来成本预测')
plt.fill_between(future_time, 
                 np.percentile(future_cost_samples, 5, axis=0), 
                 np.percentile(future_cost_samples, 95, axis=0), 
                 color='red', alpha=0.3)
plt.xlabel('时间')
plt.ylabel('成本')
plt.legend()

# 画出价格的预测
plt.subplot(1, 2, 2)
plt.plot(history_time, history_price, 'g', label='历史价格')
plt.plot(future_time, future_price_samples.mean(axis=0), 'r--', label='未来价格预测')
plt.fill_between(future_time, 
                 np.percentile(future_price_samples, 5, axis=0), 
                 np.percentile(future_price_samples, 95, axis=0), 
                 color='red', alpha=0.3)
plt.xlabel('时间')
plt.ylabel('价格')
plt.legend()

plt.tight_layout()
plt.show()


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_cost, beta_cost, gamma_cost, alpha_price, beta_price]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 40 seconds.
Sampling: [cost_obs, price_obs]


Output()

KeyError: 'future_cost_pred'